In [1]:
import copy

import numpy as np
import torch
import tqdm

import matplotlib.pyplot as plt 

# from ray import tune

import stable_baselines3
from stable_baselines3.common import vec_env, monitor


from rl_utils import lr_Environment, make_observation
from lr_utils import minimize_custom, norm_function, rosen_function

/home/tidianes/miniconda3/envs/tid_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x_dim = 1 # dimension of the problem
objective_function = norm_function
x_0 = 0.0#torch.tensor([0.0], requires_grad=True)
optimizer_class = torch.optim.SGD #torch.optim.Adam,
precision = 1e-6

In [3]:
minimize_custom(objective=objective_function,
                optimizer_class=optimizer_class,
                x_0=x_0,#0.,#[1.3, 0.7, 0.8, 1.9, 1.2],
                lr=1e-2,
                steps=3)

([0.019999999552965164, 0.03959999978542328, 0.05880799889564514],
 [0.9604000008761883, 0.922368160412159, 0.8858423829428199])

In [4]:
norm_dataset = [norm_function for _ in range(90)]
norm_env = lr_Environment(norm_dataset, num_steps=40, history_len=25)

-0.07844541073241888


In [5]:
from stable_baselines3.common.env_checker import check_env
check_env(norm_env, warn=True)

-0.7131484739612176


/home/tidianes/miniconda3/envs/tid_env/lib/python3.10/site-packages/stable_baselines3/common/env_checker.py:130: UserWarning: Your observation  has an unconventional shape (neither an image, nor a 1D vector). We recommend you to flatten the observation to have only a 1D vector or use a custom policy to properly process the data.
  warnings.warn(


In [6]:
"""
norm_env = vec_env.DummyVecEnv([
    lambda: monitor.Monitor(
        lr_Environment(norm_function, num_steps=40, history_len=25)
    )
]*32)
"""

norm_policy = stable_baselines3.PPO('MlpPolicy', norm_env, n_steps=2, verbose=0,
                                         tensorboard_log='tb_logs/norm')

/home/tidianes/miniconda3/envs/tid_env/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:151: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 2
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2 and n_envs=1)
  warnings.warn(


In [7]:
total_timesteps = 1000 #20*40*90

#quadratic_policy.learn(total_timesteps=20 * quadratic_env.envs[0].num_steps * len(norm_dataset))
norm_policy.learn(total_timesteps=total_timesteps)

-0.37193696013476235
0.6282094179192989
-0.12306061526243421
0.12822276013454356
-1.3962438834559574
-1.3788632605374942
-0.2848139659451785
1.4634918168198348
1.3337478870933546
0.18357046192797652
-2.668042346939855
-0.14449272427753254
1.4113482250793854
0.3081771331219661
-1.1555765373475988
-0.42454029445311303
0.43245584571249307
1.2558896520628196
-1.003950188442902
-0.13332534745717248
-0.7744835498382064
-0.36442148859335965
-1.4081922421952433
-1.1357559692419805
1.2077401486295967


In [8]:
#!tensorboard --logdir tb_logs

In [9]:
# Test the trained agent
actions = []
obs = norm_env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = norm_policy.predict(obs, deterministic=True)
  actions.append(action)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = norm_env.step(action)
  #print('obs=', obs, 'reward=', reward, 'done=', done)
  print('reward=', reward)
  #norm_env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

-1.171675090980155
Step 1
Action:  3
reward= -3.823715925216675
Step 2
Action:  3
reward= -3.100141763687134
Step 3
Action:  3
reward= -2.5134918689727783
Step 4
Action:  3
reward= -2.037855625152588
Step 5
Action:  3
reward= -1.652225375175476
Step 6
Action:  3
reward= -1.3395695686340332
Step 7
Action:  3
reward= -1.086078405380249
Step 8
Action:  3
reward= -0.8805561065673828
Step 9
Action:  3
reward= -0.7139257192611694
Step 10
Action:  3
reward= -0.5788271427154541
Step 11
Action:  3
reward= -0.46929383277893066
Step 12
Action:  3
reward= -0.3804878294467926
Step 13
Action:  3
reward= -0.3084869086742401
Step 14
Action:  3
reward= -0.2501108646392822
Step 15
Action:  3
reward= -0.20278161764144897
Step 16
Action:  3
reward= -0.1644085943698883
Step 17
Action:  3
reward= -0.13329702615737915
Step 18
Action:  3
reward= -0.10807280987501144
Step 19
Action:  3
reward= -0.08762182295322418
Step 20
Action:  3
reward= -0.07104086875915527
